We assume that the user has access to a data file which, in addition to the columns from the experiment, contains the state of the cell (inactive, active, peak). Further analyses are performed globally to determine how the signal strength interacts with the cell environment. The analyses are based on the ERK signal.

In [1]:
import gc
import numpy as np
import pandas as pd
import multiprocessing as mp
import plotly.express as px

from tqdm.notebook import tqdm
from scipy.spatial import KDTree
from typing import Union, List, Dict

In [3]:
RADIUS = 45.0
data = pd.read_csv("/content/data.csv")
data.head()

,Unnamed: 0,Exp_ID,Image_Metadata_Site,track_id,Image_Metadata_T,Nuclear_size,ERKKTR_ratio,FoxO3A_ratio,objNuclei_Location_Center_X,objNuclei_Location_Center_Y,Time_min,Mutation,state,is_active,Num_neighbors_active,Num_neighbors_all
0,11542490,3,1,1,0,351.0,0.8150,1.432,94.7806,86.7322,0,1,inactive,False,0,5
1,11542491,3,1,1,1,349.0,0.8096,1.409,94.9628,87.4183,5,1,inactive,False,0,5
2,11542492,3,1,1,2,354.0,0.8125,1.398,94.8870,88.0876,10,1,inactive,False,0,6
3,11542493,3,1,1,3,347.0,0.8027,1.369,95.3487,87.7550,15,1,inactive,False,0,6
4,11542494,3,1,1,4,352.0,0.8003,1.358,94.4062,87.9034,20,1,inactive,False,0,6


In [ ]:
def compute_mutation_propagation(args):
    """ Function that counts the average active neighbours rate
    for a given DataFrame using KDTree for a given
    radius and time lags (deltas).
    """
    mutation, df_full, radius, deltas = args
    df_mut = df_full[df_full['Mutation'] == mutation]
    records = []

    for (exp_id, site_id), df_ex in df_mut.groupby(['Exp_ID', 'Image_Metadata_Site']):
        for t0, df_peak in df_ex[df_ex['state'] == 'peak'].groupby('Time_min'):
            df_frame = df_ex[df_ex['Time_min'] == t0]
            if df_frame.empty:
                continue

            coords_frame = df_frame[['objNuclei_Location_Center_X', 'objNuclei_Location_Center_Y']].values
            tree_frame = KDTree(coords_frame)

            for idx, cell in df_peak.iterrows():
                pt = [cell['objNuclei_Location_Center_X'], cell['objNuclei_Location_Center_Y']]
                neigh_idxs = tree_frame.query_ball_point(pt, r=radius)
                neigh_idxs = [i for i in neigh_idxs if df_frame.index[i] != idx]
                neigh_ids = df_frame.iloc[neigh_idxs]['track_id'].values
                if len(neigh_ids) == 0:
                    continue

                for delta in deltas:
                    t1 = t0 + delta
                    df_t1 = df_ex[(df_ex['Time_min'] == t1) & (df_ex['track_id'].isin(neigh_ids))]
                    if df_t1.empty:
                        continue

                    p_active = df_t1['is_active'].mean()
                    records.append({
                        'Mutation': mutation,
                        'Delta': delta,
                        'p_active': p_active
                    })
    return records

def compute_local_propagation(
    df: pd.DataFrame,
    radius: float = RADIUS,
    deltas: Union[int, List[int]] = [-10, -5, 0, 5, 10],
    n_jobs: int = mp.cpu_count()) -> pd.DataFrame:
    """ Function counts the ratio of active neighbours in the neighbourhood
    of cells in the peak state at each time offset, using multiprocess
    to parallelize the calculations and tqdm to track progress.
    """
    if not isinstance(deltas, list):
        deltas = [deltas]

    muts = df['Mutation'].unique().tolist()
    args = [(mut, df, radius, deltas) for mut in muts]
    all_probs = []

    with mp.Pool(n_jobs) as pool:
        for prob in tqdm(pool.imap_unordered(compute_mutation_propagation, args), total=len(args), desc='Mutations'):
            all_probs.extend(prob)
    return pd.DataFrame(all_probs)

In [ ]:
mutations = compute_local_propagation(data, RADIUS, [-10, -5, 0, 5, 10])
print(mutations.groupby(['Mutation', 'Delta'])['p_active'].mean())

Mutations: 100%|██████████| 5/5 [35:21<00:00, 424.26s/it]


Mutation  Delta
1         -10      0.330461
          -5       0.348302
           0       0.360311
           5       0.359595
           10      0.353358
2         -10      0.311834
          -5       0.324416
           0       0.334585
           5       0.332066
           10      0.326430
3         -10      0.355003
          -5       0.380539
           0       0.397113
           5       0.393239
           10      0.379882
4         -10      0.547867
          -5       0.590326
           0       0.612869
           5       0.611134
           10      0.592409
5         -10      0.369458
          -5       0.388881
           0       0.402137
           5       0.400029
           10      0.388992
Name: p_active, dtype: float64


In [30]:
def compute_conditional_probability_by_mutation(args) -> Dict[str, float]:
    """
    For given mutation, compute conditional
    P(cell active at t | neighbor in state S at t-delta)
    averaged over all cells and deltas.
    """
    mutation, df_full, radius, deltas = args
    df_mut = df_full[df_full['Mutation'] == mutation]
    records = []

    for (exp_id, site_id), df_ex in df_mut.groupby(['Exp_ID','Image_Metadata_Site']):
        df_ex = df_ex.sort_values(['track_id','Time_min'])
        for delta in deltas:
            # Build index
            df_minus = df_ex.copy()
            df_minus['Time_key'] = df_minus['Time_min'] + delta
            for t, df_t in df_ex.groupby('Time_min'):
                gc.collect()
                df_prev = df_minus[df_minus['Time_key'] == t]
                if df_prev.empty:
                    continue

                coords_prev = df_prev[['objNuclei_Location_Center_X','objNuclei_Location_Center_Y']].values
                tree_prev = KDTree(coords_prev)
                coords_t = df_t[['objNuclei_Location_Center_X','objNuclei_Location_Center_Y']].values
                neigh_idxs = tree_prev.query_ball_point(coords_t, r=radius)
                for i, idx in enumerate(df_t.index):
                    st_list = df_prev.iloc[neigh_idxs[i]]['state'].values
                    if st_list.size == 0:
                        continue

                    for neigh_state in ['inactive','active','peak']:
                        if np.any(st_list == neigh_state):
                            p_active = df_t.loc[idx,'is_active']
                            records.append({
                                'Mutation': mutation,
                                'Delta': delta,
                                'neighbor_state': neigh_state,
                                'is_active': p_active
                            })
    # Aggregate
    df_rec = pd.DataFrame(records)
    if df_rec.empty:
        return pd.DataFrame()
    summary = df_rec.groupby(['Mutation','Delta','neighbor_state'])['is_active'].mean().reset_index()
    summary.rename(columns={'is_active':'p_active'}, inplace=True)
    return summary


def conditional_probabilities_all(df: pd.DataFrame,
                                  radius: float = RADIUS,
                                  deltas: list = [-5, 0, 5],
                                  n_jobs: int = mp.cpu_count()) -> pd.DataFrame:
    """ Function calculates the probability that a cell is active
    if there is a cell with the given state in its neighbourhood,
    using multiprocess to parallelize the calculations
    and tqdm to track progress.
    """
    muts = df['Mutation'].unique().tolist()
    args = [(mut, df, radius, deltas) for mut in muts]
    results = []

    with mp.Pool(n_jobs) as pool:
        for res in tqdm(pool.imap_unordered(compute_conditional_probability_by_mutation, args),
                         total=len(args), desc='Mutations'):
            if not res.empty:
                results.append(res)

    if not results:
        return pd.DataFrame()

    return pd.concat(results, ignore_index=True)


def compute_conditional_probabilities_norm(df_summary: pd.DataFrame) -> pd.DataFrame:
    """
    Normalize p_active across neighbor_state so that for each Mutation and Delta,
    sum(p_active(s)) = 1.
    """
    df = df_summary.copy()
    df['p_raw'] = df['p_active']
    df = df.copy()
    df['p_norm'] = df.groupby(['Mutation','Delta'])['p_raw']\
                 .transform(lambda x: x / x.sum())
    return df


def plot_conditional_probabilities(df_summary: pd.DataFrame) -> None:
    """ Function draws conditional probabilities of active cells
    depends on lag in time and grouped by mutations.
    """
    fig = px.line(
        df_summary,
        x='Delta',
        y='p_norm',
        color='neighbor_state',
        facet_col='Mutation',
        facet_col_wrap=3,
        markers=True,
        width=0,
        labels={
            'p_norm': 'P(cell active | neighbor state)',
            'Delta': 'Lag [min]'
        },
        hover_data=['neighbor_state'],
        title='Conditional activation probability by neighbor state and mutation'
    )

    nrows = int(np.ceil(df_summary['Mutation'].nunique() / 3))
    fig.update_layout(height=300 * nrows)
    fig.show()

In [24]:
df_conditional = conditional_probabilities_all(data[data['Image_Metadata_Site'].isin([1, 5, 9, 13, 17])])
df_conditional = compute_conditional_probabilities_norm(df_conditional)

Mutations:   0%|          | 0/5 [00:00<?, ?it/s]

In [31]:
plot_conditional_probabilities(df_conditional)

In [20]:
def compute_movement_propagation(args):
    """ Function that counts the average distance change
    for a given DataFrame using KDTree for a given
    radius and time lags.
    """
    mutation, df_full, radius, deltas = args
    df_mut = df_full[df_full['Mutation'] == mutation]
    records = []

    for (exp_id, site_id), df_ex in df_mut.groupby(['Exp_ID',
                                                    'Image_Metadata_Site']):
        df_ex = df_ex.sort_values(['track_id','Time_min'])
        for t0, df_t0 in df_ex.groupby('Time_min'):
            # positions of all cells and of peaks at t0
            coords_all = df_t0[['objNuclei_Location_Center_X',
                                'objNuclei_Location_Center_Y']].values
            peak_positions = df_t0[df_t0['state'] == 'peak']\
                            [['objNuclei_Location_Center_X',
                              'objNuclei_Location_Center_Y']]\
                            .values
            if peak_positions.size == 0:
                continue

            tree_peaks = KDTree(peak_positions)
            # minimum distance at t0
            d0 = tree_peaks.query(coords_all)[0]
            for delta in deltas:
                gc.collect()
                t1 = t0 + delta
                df_t1 = df_ex[df_ex['Time_min']==t1]
                if df_t1.empty:
                    continue

                coords_all1 = df_t1[['objNuclei_Location_Center_X',
                                     'objNuclei_Location_Center_Y']].values
                tree_peaks1 = KDTree(peak_positions)
                d1 = tree_peaks1.query(coords_all1)[0]
                # align by track_id
                merged = pd.DataFrame({
                    'track_id': df_t0['track_id'].values,
                    'd0': d0
                }).merge(
                    pd.DataFrame({
                        'track_id': df_t1['track_id'].values,
                        'd1': d1
                    }), on='track_id', how='inner'
                )
                if merged.empty:
                    continue
                mean_delta_d = (merged['d1'] - merged['d0']).mean()
                records.append({
                    'Mutation': mutation,
                    'Delta': delta,
                    'mean_distance_change': mean_delta_d
                })

    return pd.DataFrame(records)


def movement_propagation_all(df: pd.DataFrame,
                             radius: float = RADIUS,
                             deltas = [-10, -5, 5, 10, 15, 20, 25],
                             n_jobs: int = mp.cpu_count()) -> pd.DataFrame:
    """ Function that counts the average change in position,
    relative to cells in the peak state, using multiprocess
    to parallelize the calculations and tqdm to track progress.
    """
    muts = df['Mutation'].unique().tolist()
    args = [(mut, df, radius, deltas) for mut in muts]
    with mp.Pool(n_jobs) as pool:
        all = list(tqdm(pool.imap_unordered(compute_movement_propagation, args),
                        total=len(args), desc='Movement'))
    return pd.concat(all, ignore_index=True)


def plot_movement_propagation(df_move: pd.DataFrame,
                              facet: bool = False) -> None:
    """
    Function draws boxplots of mean_distance_change vs. Delta,
    grouped by Mutation. If facet=True, creates one facet per Mutation;
    otherwise colors boxes by Mutation.
    """
    if facet:
        fig = px.box(
            df_move,
            x='Delta',
            y='mean_distance_change',
            facet_col='Mutation',
            facet_col_wrap=3,
            title='Distance in movement propagation',
            labels={
                'mean_distance_change': 'distance [µm]',
                'Delta': 'Lag [min]'
            }
        )
    else:
        fig = px.box(
            df_move,
            x='Delta',
            y='mean_distance_change',
            color='Mutation',
            title='Distance in movement propagation',
            labels={
                'mean_distance_change': 'distance [µm]',
                'Delta': 'Lag [min]'
            }
        )
    fig.update_layout(height=400 if not facet else\
                      300*int(np.ceil(df_move['Mutation'].nunique()/3)))
    fig.show()

In [17]:
move = movement_propagation_all(data)

Movement:   0%|          | 0/5 [00:00<?, ?it/s]

In [21]:
plot_movement_propagation(move, facet=False)

In [22]:
plot_movement_propagation(move, facet=True)